In [1]:
# 把学生的平均分计算出来

In [1]:
import jutil
import pandas as pd
import numpy as np

## 学院分析

In [2]:
def load_college_students_df(college_code):
    sql = "select *,CONVERT(VARCHAR,birthday,112) as birth from view_stu_course_mark where college_code='%s' and grade='2014'" % college_code
    return jutil.load_pd_df(sql)

In [3]:
# 信息学院
df = load_college_students_df('04')
df.head()

,birth,birthday,city,class_code,class_name,college_code,college_name,course_code,course_name,course_type,...,province,sex,speciality_code,speciality_name,student_id,teach_class_id,teacher,term_id,term_name,term_order
0,19960615,834768000000,北京,0403201403,通信2014-03班,04,信息科学与技术学院,0471006,计算机组成原理C,必,...,北京,男,0403,通信工程,20122169,3,唐慧佳,79,2016-2017第1学期,3
1,19960615,834768000000,北京,0403201403,通信2014-03班,04,信息科学与技术学院,0471007,现代交换原理,限,...,北京,男,0403,通信工程,20122169,2,刘恒,72,2014-2015第2学期,2
2,19960615,834768000000,北京,0403201403,通信2014-03班,04,信息科学与技术学院,0473001,DSP原理与应用,限,...,北京,男,0403,通信工程,20122169,1,邹应全,75,2015-2016第1学期,2
3,19960615,834768000000,北京,0403201403,通信2014-03班,04,信息科学与技术学院,1271002,随机过程C,必,...,北京,男,0403,通信工程,20122169,1,彭皓,73,2014-2015第1学期补考,1
4,19960615,834768000000,北京,0403201403,通信2014-03班,04,信息科学与技术学院,3006654,电路分析BⅠ,必,...,北京,男,0403,通信工程,20122169,4,谭永霞,64,2012-2013第2学期,0


In [4]:
# 要根据学生id聚类然后计算其平均分
def get_avg_stu_marks_df(df):
    groupd  = df.groupby(['student_id','college_code','college_name','sex','name','birth','class_code','class_name','nation_name',
        'party','province','city','speciality_code','speciality_name','grade'],as_index=False) # 保留列
    groupd = groupd.agg({'pmark':np.average})
    return groupd

In [5]:
marks = get_avg_stu_marks_df(df)
marks.head()

,student_id,college_code,college_name,sex,name,birth,class_code,class_name,nation_name,party,province,city,speciality_code,speciality_name,grade,pmark
0,20122169,04,信息科学与技术学院,男,王钧毅,19960615,0403201403,通信2014-03班,汉族,中国共产主义青年团团员,北京,北京,0403,通信工程,2014,53.827358
1,20122639,04,信息科学与技术学院,男,杨永虎,19920702,0408201402,软件2014-02班,汉族,中国共产党预备党员,山东,聊城市,0408,软件工程,2014,84.775641
2,20130068,04,信息科学与技术学院,男,任添笑,19950615,0402201401,计算机2014-01班,汉族,中国共产主义青年团团员,陕西,西安市,0402,计算机科学与技术,2014,81.977778
3,20131649,04,信息科学与技术学院,男,王鑫,19940202,0406201401,自动2014-01班,汉族,中国共产主义青年团团员,重庆,重庆市,0406,自动化,2014,61.821591
4,20131664,04,信息科学与技术学院,男,盛守彬,19940208,0406201402,自动2014-02班,汉族,中国共产主义青年团团员,辽宁,本溪市,0406,自动化,2014,55.774000


In [31]:
sex_cnt = marks['sex'].value_counts().to_dict()
type(int(sex_cnt['男']))

int

In [34]:
# 计算学院男女的平均成绩
g = marks.groupby('sex',as_index=False).agg({'pmark':np.average})
dg = dict(g)
type(dg['pmark'][0].item())

float

In [36]:
type(marks['pmark'].mean().item())

float

In [24]:
# 计算学院学霸和学渣人数
g = marks[marks['pmark']>90].groupby('sex',as_index=False).agg({'pmark':'count'}).to_dict()
# dict(g)
len(g['sex'])

2

In [23]:
g = marks[marks['pmark']<65].groupby('sex',as_index=False).agg({'pmark':'count'}).to_dict()

In [25]:
# 计算学院挂科人数
g = df[df['pmark']<60].drop_duplicates(subset=['student_id']).groupby('sex',as_index=False).agg({'pmark':'count'}).to_dict()
g

{'pmark': {0: 74, 1: 342}, 'sex': {0: '女', 1: '男'}}

## 特别条件成绩分析

In [57]:
def get_constellation(row):
    birth = str(row['birth'])[4:]
    if (birth>='1222' and birth<='1231') or (birth>='0101' and birth<='0119') :
        return '魔羯座'
    elif birth>='0120' and birth<='0218':
        return '水瓶座'
    elif birth>='0217' and birth<='0320':
        return '双鱼座'
    elif birth>='0319' and birth<='0420':
        return '牡羊座'
    elif birth>='0419' and birth<='0520':
        return '金牛座'
    elif birth>='0519' and birth<='0621':
        return '双子座'
    elif birth>='0620' and birth<='0722':
        return '巨蟹座'
    elif birth>='0721' and birth<='0822':
        return '狮子座'
    elif birth>='0821' and birth<='0922':
        return '处女座'
    elif birth>='0921' and birth<='1022':
        return '天秤座'
    elif birth>='1023' and birth<='1121':
        return '天蝎座'
    elif birth>='1122' and birth<='1221':
        return '射手座'
    else:
        return '未知星座'

In [58]:
# 按星座
marks['constellation'] = marks.apply(get_constellation,axis=1)
marks.head()

,student_id,college_code,college_name,sex,name,birth,class_code,class_name,nation_name,party,province,city,speciality_code,speciality_name,grade,pmark,constellation
0,20122169,04,信息科学与技术学院,男,王钧毅,19960615,0403201403,通信2014-03班,汉族,中国共产主义青年团团员,北京,北京,0403,通信工程,2014,53.827358,双子座
1,20122639,04,信息科学与技术学院,男,杨永虎,19920702,0408201402,软件2014-02班,汉族,中国共产党预备党员,山东,聊城市,0408,软件工程,2014,84.775641,巨蟹座
2,20130068,04,信息科学与技术学院,男,任添笑,19950615,0402201401,计算机2014-01班,汉族,中国共产主义青年团团员,陕西,西安市,0402,计算机科学与技术,2014,81.977778,双子座
3,20131649,04,信息科学与技术学院,男,王鑫,19940202,0406201401,自动2014-01班,汉族,中国共产主义青年团团员,重庆,重庆市,0406,自动化,2014,61.821591,水瓶座
4,20131664,04,信息科学与技术学院,男,盛守彬,19940208,0406201402,自动2014-02班,汉族,中国共产主义青年团团员,辽宁,本溪市,0406,自动化,2014,55.774000,水瓶座


In [66]:
marks.groupby(['constellation','sex'],as_index=False).agg({'pmark':'mean'})

,constellation,sex,pmark
0,双子座,女,80.835060
1,双子座,男,77.650870
2,双鱼座,女,80.193352
3,双鱼座,男,77.695130
4,处女座,女,83.505883
5,处女座,男,76.375345
6,天秤座,女,83.687006
7,天秤座,男,76.993231
8,天蝎座,女,83.015381
9,天蝎座,男,77.982852


In [69]:
marks.groupby('province',as_index=False).agg({'pmark':'mean'})

,province,pmark
0,上海,74.507950
1,云南,72.813413
2,内蒙古,73.494827
3,北京,70.552434
4,北京市,87.236585
5,吉林,74.904753
6,四川,78.326618
7,四川省,77.880769
8,天津,77.548063
9,宁夏,79.875576
